## Import Packages

In [1]:
!pip install geopy

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [2]:
!pip install folium

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


/Users/huangyihuan/anaconda2/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


## 1. Download and Explore Dataset

In [4]:
# getting dat using BeautifulSoup packages
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 

In [5]:
# read in data
df = pd.read_html(str(table))

In [6]:
# get rid of non-assigned borough rows
toronto_data = df[0]
toronto_data = toronto_data[toronto_data['Borough'] != 'Not assigned']
toronto_data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
# checking whether neighbourhood has any non-assigned values
ind = toronto_data[toronto_data['Neighbourhood'] == 'Not assigned'].index
# change them accordingly
toronto_data.iloc[ind,2] = toronto_data.iloc[ind,1]

/Users/huangyihuan/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/huangyihuan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [8]:
toronto_data.shape

(211, 3)

In [9]:
toronto_data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## 2. Get location details for each neighborhood

In [10]:
geocode = pd.read_csv('Geospatial_Coordinates.csv')
geocode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
toronto_data_geo = pd.merge(toronto_data, geocode, how = 'left', left_on = toronto_data.Postcode, right_on = geocode['Postal Code'])
toronto_data_geo.drop(['key_0'], axis = 1, inplace = True)
toronto_data_geo.drop(['Postal Code'], axis = 1, inplace = True)
toronto_data_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


## 3. Clustering and Segmenting

### clean dataset to include only dt Toronto info

In [12]:
toronto_data_geo['Borough'].value_counts()

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: Borough, dtype: int64

In [13]:
dt_toronto = toronto_data_geo[toronto_data_geo['Borough'] == 'Downtown Toronto']

In [14]:
dt_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [15]:
!pip install geocoder

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [16]:
address = 'Downtown Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451.


### Create a map of dt Toronto with neighborhoods superimposed on top.

In [17]:
# create map of Downtown Toronto using latitude and longitude values
map_dtToronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(dt_toronto['Latitude'], dt_toronto['Longitude'], dt_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtToronto)  
    
map_dtToronto

### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'S4JSAPBEB2HWJ1T2HPR3BJX1PVGDIAA30G5C1151VRNGCRZX' # your Foursquare ID
CLIENT_SECRET = 'KMM3DCK4WLTODUAOILM2K0ZCSFLDXM3B3DA52QLU0BPTAPKA' # your Foursquare Secret
VERSION = '20191009' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S4JSAPBEB2HWJ1T2HPR3BJX1PVGDIAA30G5C1151VRNGCRZX
CLIENT_SECRET:KMM3DCK4WLTODUAOILM2K0ZCSFLDXM3B3DA52QLU0BPTAPKA


### Explore neighborhoods in dt Toronto

In [19]:
# Get API from foursquares and transform them into dataframes
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
LIMIT = 100
dtToronto_venues = getNearbyVenues(names=dt_toronto['Neighbourhood'],
                                   latitudes=dt_toronto['Latitude'],
                                   longitudes=dt_toronto['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [21]:
dtToronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


### Analyze each neighborhood

In [23]:
# one hot encoding
dtToronto_onehot = pd.get_dummies(dtToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dtToronto_onehot['Neighborhood'] = dtToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = list(dtToronto_onehot.columns)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
dtToronto_onehot = dtToronto_onehot[cols]

dtToronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
dtToronto_grouped = dtToronto_onehot.groupby('Neighborhood').mean().reset_index()
dtToronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Adelaide,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.0,0.0,0.010000,0.01,0.0,0.03,0.000000,0.0,0.000000,0.010000,0.000000,0.0400,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.01,0.0000,0.01,0.020000,0.0,0.0,0.01,0.03,0.01,0.000000,0.050000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.080000,0.0,0.0,0.0,0.01,0.000000,0.0,0.020000,0.0,0.020000,0.000000,0.0,0.010000,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.0,0.020000,0.0,0.000000,0.01,0.0,0.000000,0.01,0.000000,0.010000,0.000000,0.02,0.01,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,0.0,0.010000,0.010000,0.0,0.000000,0.010000,0.010000,0.010000,0.000000,0.0,0.0,0.0,0.01,0.01,0.000000,0.020000,0.000000,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.0,0.01,0.000000,0.01,0.010000,0.01,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.020000,0.0000,0.000000,0.01,0.01,0.0,0.0,0.0,0.000000,0.0,0.01,0.030000,0.0,0.0,0.01,0.01,0.010000,0.0,0.0000,0.010000,0.0,0.000000,0.0,0.

return a dataframe with 10 most common venues in that neighborhood

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dtToronto_grouped['Neighborhood']

for ind in np.arange(dtToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,American Restaurant,Thai Restaurant,Restaurant,Asian Restaurant,Sushi Restaurant,Burger Joint,Hotel
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden
2,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Cheese Shop,Steakhouse,Seafood Restaurant,Farmers Market,Beer Bar,Belgian Restaurant
3,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden
4,Cabbagetown,Coffee Shop,Pizza Place,Restaurant,Café,Italian Restaurant,Market,Park,Bakery,Pub,General Entertainment


### Clustering

In [89]:
# set number of clusters
kclusters = 5

dtToronto_grouped_clustering = dtToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 1, 0, 0, 2, 4, 0, 0], dtype=int32)

In [90]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtToronto_merged = dt_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dtToronto_merged = dtToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

dtToronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Historic Site
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Historic Site
13,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Fast Food Restaurant,Bakery,Bubble Tea Shop,Sporting Goods Shop
14,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Fast Food Restaurant,Bakery,Bubble Tea Shop,Sporting Goods Shop
27,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Pizza Place,Bakery,Hotel,Park,Breakfast Spot,Clothing Store


In [91]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtToronto_merged['Latitude'], dtToronto_merged['Longitude'], dtToronto_merged['Neighbourhood'], dtToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 4. Examining clusters

In [92]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 0, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Historic Site
3,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Historic Site
13,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Fast Food Restaurant,Bakery,Bubble Tea Shop,Sporting Goods Shop
14,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Café,Fast Food Restaurant,Bakery,Bubble Tea Shop,Sporting Goods Shop
27,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Italian Restaurant,Pizza Place,Bakery,Hotel,Park,Breakfast Spot,Clothing Store
37,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Café,Cheese Shop,Steakhouse,Seafood Restaurant,Farmers Market,Beer Bar,Belgian Restaurant
41,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Sandwich Place,Ice Cream Shop,Spa,Chinese Restaurant,Middle Eastern Restaurant,Bar
49,Downtown Toronto,0,Coffee Shop,Café,Bar,American Restaurant,Thai Restaurant,Restaurant,Asian Restaurant,Sushi Restaurant,Burger Joint,Hotel
50,Downtown Toronto,0,Coffee Shop,Café,Bar,American Restaurant,Thai Restaurant,Restaurant,Asian Restaurant,Sushi Restaurant,Burger Joint,Hotel
51,Downtown Toronto,0,Coffee Shop,Café,Bar,American Restaurant,Thai Restaurant,Restaurant,Asian Restaurant,Sushi Restaurant,Burger Joint,Hotel


In [93]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 1, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
164,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden
165,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden
166,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden
167,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden
168,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden
169,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden
170,Downtown Toronto,1,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Bar,Plane,Sculpture Garden


In [94]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 2, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
143,Downtown Toronto,2,Café,Bookstore,Japanese Restaurant,Italian Restaurant,Bar,Bakery,Restaurant,Nightclub,Chinese Restaurant,Pub
144,Downtown Toronto,2,Café,Bookstore,Japanese Restaurant,Italian Restaurant,Bar,Bakery,Restaurant,Nightclub,Chinese Restaurant,Pub
152,Downtown Toronto,2,Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Bakery,Arts & Crafts Store
153,Downtown Toronto,2,Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Bakery,Arts & Crafts Store
154,Downtown Toronto,2,Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Coffee Shop,Mexican Restaurant,Dumpling Restaurant,Bakery,Arts & Crafts Store


In [95]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 3, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
183,Downtown Toronto,3,Park,Playground,Trail,Building,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


In [96]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 4, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Downtown Toronto,4,Café,Grocery Store,Park,Restaurant,Coffee Shop,Convenience Store,Nightclub,Diner,Baby Store,Italian Restaurant
